In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Daily swing-trade scheduler for Indian equities (EMA/RSI/ADX + VOLAR + MVO).
- Best params baked in:
    ema_fast=10, ema_slow=50, rsi_buy_max=35, adx_min=25, target_pct=7%, within_52w=50%, top_k=5
- Uses TODAY's EOD bar as signal_date; schedules execution for next trading day (planned_exec_date).
- Writes:
    plans/plan_YYYYMMDD_HHMMSS.csv        --> SELL/BUY actions for T+1
    plans/candidates_YYYYMMDD_HHMMSS.csv  --> full ranked candidate list (audit)
- Maintains:
    data/positions.csv (virtual ledger of holdings)
- Optional Telegram alert.

Run this after market close (e.g., ~18:00 IST).
"""

import os, json, math, logging, warnings
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple

import numpy as np
import pandas as pd
import requests
from pandas.tseries.offsets import BDay

warnings.filterwarnings("ignore", category=FutureWarning)

try:
    import yfinance as yf
except Exception:
    raise SystemExit("Please install yfinance: pip install yfinance")

# --------------------------
# LOGGING
# --------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("daily_sched")

# --------------------------
# CONFIG (your best params)
# --------------------------
@dataclass
class Config:
    # Dates
    lookback_start: str = "2019-01-01"
    end_date: Optional[str] = None  # None => today

    # Universe (either set list here, or provide a file path with one symbol per line)
    static_symbols: List[str] = field(default_factory=list)  # empty => use file
    static_symbols_path: str = "universe.txt"

    # IO
    cache_dir: str = "cache"
    data_dir: str  = "data"
    out_dir: str   = "plans"
    positions_csv: str = "data/positions.csv"   # persistent holdings (virtual)

    # Fees
    apply_fees: bool = True

    # Capital & portfolio
    initial_capital_inr: float = 200_000.0
    max_concurrent_positions: int = 5
    deploy_cash_frac: float = 0.25  # cap daily deploy to % of available cash

    # Indicators (best params)
    ema_fast: int = 10
    ema_slow: int = 50
    ema_htf: int  = 200
    use_htf_trend: bool = True   # require Close>EMA200
    rsi_len: int = 14
    rsi_buy_max: float = 35.0    # RSI cross up through this
    rsi_sell_min: float = 70.0
    adx_len: int = 14
    adx_min: float = 25.0        # stronger trend

    # Exits (fixed stop/target)
    stop_loss_pct: float = 0.05
    target_pct: float    = 0.07

    # Ranking / Filters
    benchmark_try: Tuple[str,...] = ("^CNX500","^CRSLDX","^NSE500","^NIFTY500","^BSE500","^NSEI")
    volar_lookback: int = 126
    filter_52w_window: int = 252
    within_pct_of_52w_high: float = 0.50
    top_k_daily: int = 5

    # Telegram alerts (optional)
    telegram_enabled: bool = False
    telegram_bot_token: str = os.getenv("TELEGRAM_BOT_TOKEN", "")
    telegram_chat_id:  str = os.getenv("TELEGRAM_CHAT_ID", "")  # e.g. "-1001234567890"

CFG = Config()

# --------------------------
# FEES (same as earlier spec)
# --------------------------
APPLY_FEES = True

def calc_fees(turnover_buy: float, turnover_sell: float) -> float:
    if not APPLY_FEES:
        return 0.0
    BROKER_PCT = 0.001
    BROKER_MIN = 5.0
    BROKER_CAP = 20.0
    STT_PCT = 0.001
    STAMP_BUY_PCT = 0.00015
    EXCH_PCT = 0.0000297
    SEBI_PCT = 0.000001
    IPFT_PCT = 0.000001
    GST_PCT = 0.18
    DP_SELL = 20.0 if turnover_sell >= 100 else 0.0

    def _broker(turnover):
        if turnover <= 0: return 0.0
        fee = turnover * BROKER_PCT
        return max(BROKER_MIN, min(fee, BROKER_CAP))

    br_buy  = _broker(turnover_buy)
    br_sell = _broker(turnover_sell)
    stt   = STT_PCT * (turnover_buy + turnover_sell)
    stamp = STAMP_BUY_PCT * turnover_buy
    exch  = EXCH_PCT * (turnover_buy + turnover_sell)
    sebi  = SEBI_PCT * (turnover_buy + turnover_sell)
    ipft  = IPFT_PCT * (turnover_buy + turnover_sell)
    dp    = DP_SELL
    gst_base = br_buy + br_sell + dp + exch + sebi + ipft
    gst   = GST_PCT * gst_base
    return float((br_buy + br_sell) + stt + stamp + exch + sebi + ipft + dp + gst)

# --------------------------
# Utils
# --------------------------
def ensure_dirs(*paths):
    for p in paths:
        os.makedirs(p, exist_ok=True)

def today_str():
    return pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y-%m-%d")

def next_trading_day(cur_dt: pd.Timestamp) -> pd.Timestamp:
    # Approximate with business day; replace with NSE holiday calendar if needed
    return (cur_dt + BDay(1)).normalize()

def load_universe(cfg: Config) -> List[str]:
    if cfg.static_symbols:  # non-empty list wins
        syms = cfg.static_symbols
    else:
        if not os.path.exists(cfg.static_symbols_path):
            raise FileNotFoundError(f"Universe file missing: {cfg.static_symbols_path}")
        with open(cfg.static_symbols_path, "r") as f:
            syms = [line.strip() for line in f if line.strip()]
    out = []
    for s in syms:
        s = s.upper()
        if not s.endswith(".NS"):
            s = f"{s}.NS"
        out.append(s)
    uniq, seen = [], set()
    for s in out:
        if s not in seen:
            uniq.append(s); seen.add(s)
    return uniq

def fetch_prices(tickers: List[str], start: str, end: Optional[str], cache_dir: str) -> Dict[str, pd.DataFrame]:
    ensure_dirs(cache_dir)
    end = end or today_str()
    data = {}
    for t in tickers:
        path = os.path.join(cache_dir, f"{t.replace('^','_')}.parquet")
        df = None
        if os.path.exists(path):
            try:
                df = pd.read_parquet(path)
            except Exception:
                df = None
        if df is None or df.empty or pd.to_datetime(df.index[-1]).strftime("%Y-%m-%d") < end:
            try:
                d = yf.download(t, start=start, end=end, auto_adjust=True, progress=False, multi_level_index=False)
                if d is None or d.empty:
                    continue
                d = d.rename(columns=str.title)[["Open","High","Low","Close","Volume"]].dropna()
                d.index.name = "date"
                d.to_parquet(path)
                df = d
            except Exception:
                continue
        data[t] = df
    return data

def ema(s: pd.Series, span: int) -> pd.Series:
    return s.ewm(span=span, adjust=False, min_periods=span).mean()

def rsi(series: pd.Series, length: int = 14) -> pd.Series:
    d = series.diff()
    gain = (d.where(d > 0, 0.0)).rolling(length).mean()
    loss = (-d.where(d < 0, 0.0)).rolling(length).mean()
    rs = gain / loss.replace(0.0, np.nan)
    return (100 - 100/(1+rs)).fillna(50)

def adx_series(high: pd.Series, low: pd.Series, close: pd.Series, length: int = 14) -> pd.Series:
    """Return ADX as a single pd.Series (no DataFrame surprises)."""
    high = pd.Series(high).astype(float)
    low  = pd.Series(low).astype(float)
    close = pd.Series(close).astype(float)
    ph, pl, pc = high.shift(1), low.shift(1), close.shift(1)
    up, dn = high - ph, pl - low
    plus_dm  = up.where((up > dn) & (up > 0), 0.0)
    minus_dm = dn.where((dn > up) & (dn > 0), 0.0)
    tr = pd.concat([(high - low).abs(), (high - pc).abs(), (low - pc).abs()], axis=1).max(axis=1)
    alpha = 1.0 / length
    atr = tr.ewm(alpha=alpha, adjust=False, min_periods=length).mean()
    plus_di  = 100 * (plus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean() / atr)
    minus_di = 100 * (minus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean() / atr)
    dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0, np.nan)
    adx_ser = dx.ewm(alpha=alpha, adjust=False, min_periods=length).mean()
    return adx_ser.astype(float)

def compute_indicators(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    out = df.copy()
    out["ema_fast"] = ema(out["Close"], cfg.ema_fast)
    out["ema_slow"] = ema(out["Close"], cfg.ema_slow)
    out["ema_htf"]  = ema(out["Close"], cfg.ema_htf)
    out["rsi"]      = rsi(out["Close"], cfg.rsi_len)
    out["rsi_prev"] = out["rsi"].shift(1)
    adx_val = adx_series(out["High"], out["Low"], out["Close"], cfg.adx_len)
    if isinstance(adx_val, pd.DataFrame):  # defensive
        adx_val = adx_val.iloc[:, 0]
    out["adx"] = pd.Series(adx_val, index=out.index).astype(float)
    out["trend_strength"] = (out["ema_fast"] - out["ema_slow"]) / out["Close"]
    out["high_52w"] = out["Close"].rolling(cfg.filter_52w_window).max()
    return out.dropna()

def pick_benchmark(benchmarks, start, end, cache_dir):
    for t in benchmarks:
        data = fetch_prices([t], start, end, cache_dir)
        df = data.get(t)
        if df is not None and not df.empty:
            log.info("Using benchmark: %s", t); return t, df
    idx = pd.date_range(start=start, end=end or today_str(), freq="B")
    return "SYNTH", pd.DataFrame({"Close": np.ones(len(idx))}, index=idx)

def volar_score(end_dt: pd.Timestamp, ser: pd.Series, bench: pd.Series, lb: int) -> float:
    s = ser.loc[:end_dt].pct_change().dropna().iloc[-lb:]
    b = bench.loc[:end_dt].pct_change().dropna().iloc[-lb:]
    common = pd.concat([s,b], axis=1).dropna()
    if common.shape[0] < max(20, int(0.4*lb)): return 0.0
    excess = common.iloc[:,0] - common.iloc[:,1]
    vol = common.iloc[:,0].std(ddof=0)
    return 0.0 if vol <= 1e-8 else float((excess.mean()/vol) * math.sqrt(252.0))

def markowitz_long_only(mu: np.ndarray, Sigma: np.ndarray) -> np.ndarray:
    n = len(mu)
    if n == 0: return np.array([])
    eps = 1e-6
    Sigma = Sigma + eps*np.eye(n)
    best_w, best_sr = np.full(n, 1.0/n), -1e9
    for lmbd in np.logspace(-3,3,31):
        active = np.ones(n, dtype=bool); w=None
        for _ in range(n):
            idx = np.where(active)[0]
            if len(idx)==0: w = np.full(n, 1.0/n); break
            S = Sigma[np.ix_(idx,idx)]
            o = np.ones(len(idx))
            m = mu[idx]
            A = np.block([[2*lmbd*S, o[:,None]],[o[None,:], np.zeros((1,1))]])
            b = np.concatenate([m, np.array([1.0])])
            try: sol = np.linalg.solve(A,b); w_sub=sol[:len(idx)]
            except np.linalg.LinAlgError: w_sub = np.full(len(idx), 1.0/len(idx))
            w = np.zeros(n); w[idx]=w_sub
            if (w<0).any():
                active[np.argmin(w)] = False
                continue
            break
        if w is None: continue
        w = np.clip(w,0,None); s=w.sum()
        if s<=0: continue
        w /= s
        mu_p = float(mu @ w); vol_p = float(np.sqrt(w @ Sigma @ w))
        if vol_p<=1e-8: continue
        sr = mu_p/vol_p
        if sr>best_sr: best_sr, best_w = sr, w.copy()
    return best_w

# --------------------------
# Positions state
# --------------------------
def load_positions(cfg: Config) -> pd.DataFrame:
    if not os.path.exists(cfg.positions_csv):
        return pd.DataFrame(columns=["ticker","entry_date","entry_price","shares"])
    df = pd.read_csv(cfg.positions_csv)
    if df.empty:
        return pd.DataFrame(columns=["ticker","entry_date","entry_price","shares"])
    df["entry_date"] = pd.to_datetime(df["entry_date"])
    return df

def save_positions(df: pd.DataFrame, cfg: Config):
    ensure_dirs(os.path.dirname(cfg.positions_csv) or ".")
    df.to_csv(cfg.positions_csv, index=False)

def current_cash(cfg: Config, positions: pd.DataFrame, latest_prices: Dict[str, float]) -> float:
    # Planning simplification: use configured capital as "available".
    # If you want realized P&L, extend with a trade ledger & equity calc.
    return float(cfg.initial_capital_inr)

# --------------------------
# Main daily plan builder
# --------------------------
def build_daily_plan(cfg: Config):
    global APPLY_FEES
    APPLY_FEES = cfg.apply_fees

    ensure_dirs(cfg.cache_dir, cfg.data_dir, cfg.out_dir)

    # Universe & data
    universe = load_universe(cfg)
    dm = fetch_prices(universe, cfg.lookback_start, cfg.end_date, cfg.cache_dir)
    bench_tkr, bench_df = pick_benchmark(cfg.benchmark_try, cfg.lookback_start, cfg.end_date, cfg.cache_dir)
    log.info("Universe size with data: %d | Benchmark: %s", len(dm), bench_tkr)

    # Prepare indicators
    ind_map = {}
    for t, df in dm.items():
        if df is None or df.empty: continue
        ind_map[t] = compute_indicators(df, cfg)

    # Load positions
    pos = load_positions(cfg)
    held = set(pos["ticker"].tolist())

    # Determine signal (today) and planned exec (T+1) using benchmark calendar
    all_dates = bench_df.index
    if len(all_dates) < 1:
        raise RuntimeError("No dates in benchmark to build a plan.")
    signal_dt = all_dates[-1]
    exec_dt   = next_trading_day(signal_dt)

    # ---------------- SELL logic (execute at T+1 open) ----------------
    sell_rows = []
    for _, row in pos.iterrows():
        t = row["ticker"]
        df = ind_map.get(t)
        pxdf = dm.get(t)
        if df is None or pxdf is None or df.empty or pxdf.empty:
            continue
        if signal_dt not in df.index or signal_dt not in pxdf.index:
            continue

        r = df.loc[signal_dt]
        indicator_exit = (r["rsi"] > cfg.rsi_sell_min) or (r["ema_fast"] < r["ema_slow"])

        entry_px = float(row["entry_price"])
        shares_held = int(row["shares"])
        stop_px  = entry_px * (1 - cfg.stop_loss_pct)
        tgt_px   = entry_px * (1 + cfg.target_pct)

        # Use TODAY’s bar to decide if stop/target was hit; execute tomorrow
        s_low, s_high = float(pxdf.loc[signal_dt, "Low"]), float(pxdf.loc[signal_dt, "High"])
        hit = None
        if s_low <= stop_px and s_high >= tgt_px:
            hit = "target"
        elif s_low <= stop_px:
            hit = "stop"
        elif s_high >= tgt_px:
            hit = "target"
        elif indicator_exit:
            hit = "indicator_exit"

        if hit:
            sell_rows.append({
                "action": "SELL",
                "ticker": t,
                "signal_date": str(signal_dt.date()),
                "planned_exec_date": str(exec_dt.date()),
                "reason": hit,                             # stop | target | indicator_exit
                "exit_qty": shares_held,                   # suggest full exit
                "ref_close_signal": float(pxdf.loc[signal_dt, "Close"]),
                "ref_stop_level": float(stop_px),
                "ref_target_level": float(tgt_px),
                "rsi": float(r["rsi"]), "adx": float(r["adx"]),
                "ema_fast": float(r["ema_fast"]), "ema_slow": float(r["ema_slow"]), "ema_htf": float(r["ema_htf"]),
            })

    # ---------------- BUY candidates (signal today, exec T+1) ----------------
    cand = []
    for t, df in ind_map.items():
        if signal_dt not in df.index:
            continue
        r = df.loc[signal_dt]
        trend_ok = (r["ema_fast"] > r["ema_slow"])
        htf_ok   = (r["Close"] > r["ema_htf"]) if cfg.use_htf_trend else True
        rsi_cross = (r["rsi_prev"] < cfg.rsi_buy_max) and (r["rsi"] >= cfg.rsi_buy_max)
        adx_ok  = (r["adx"] > cfg.adx_min)
        if trend_ok and htf_ok and rsi_cross and adx_ok:
            pxdf = dm[t]
            if signal_dt not in pxdf.index:
                continue
            close_signal = float(pxdf.loc[signal_dt, "Close"])
            hist = pxdf["Close"].loc[:signal_dt]
            window = hist.iloc[-cfg.filter_52w_window:] if len(hist)>=cfg.filter_52w_window else hist
            high_52w = float(window.max()) if len(window) else np.nan
            if not (high_52w and high_52w > 0):
                continue
            if close_signal < cfg.within_pct_of_52w_high * high_52w:
                continue
            volar = volar_score(signal_dt, pxdf["Close"], bench_df["Close"], cfg.volar_lookback)
            cand.append({
                "ticker": t,
                "close_signal": close_signal,
                "rsi": float(r["rsi"]), "adx": float(r["adx"]),
                "ema_fast": float(r["ema_fast"]), "ema_slow": float(r["ema_slow"]), "ema_htf": float(r["ema_htf"]),
                "high_52w": float(high_52w),
                "volar": float(volar),
                "signal_reason": "RSI↑35; EMA10>EMA50; Close>EMA200; ADX>25"
            })

    # Remove already-held tickers
    if cand:
        cand = [c for c in cand if c["ticker"] not in held]

    # Rank by VOLAR
    cand = sorted(cand, key=lambda x: x["volar"], reverse=True)

    # --- Build full ranked candidates table (audit) ---
    cand_ranked = []
    for i, c in enumerate(cand, start=1):
        pct_52w = c["close_signal"] / c["high_52w"] if c["high_52w"] > 0 else np.nan
        cand_ranked.append({
            "rank": i,
            "ticker": c["ticker"],
            "signal_date": str(signal_dt.date()),
            "planned_exec_date": str(exec_dt.date()),
            "close_signal": float(c["close_signal"]),
            "rsi": float(c["rsi"]),
            "adx": float(c["adx"]),
            "ema_fast": float(c["ema_fast"]),
            "ema_slow": float(c["ema_slow"]),
            "ema_htf": float(c["ema_htf"]),
            "high_52w": float(c["high_52w"]),
            "pct_of_52w_high": float(pct_52w),
            "volar": float(c["volar"]),
            "signal_reason": c["signal_reason"],
        })

    # Capacity AFTER executing sells
    held_after_sells = len(held) - len({s["ticker"] for s in sell_rows})
    capacity = max(0, cfg.max_concurrent_positions - held_after_sells)
    slots = min(capacity, cfg.top_k_daily)
    buy_sel = cand[:max(0, slots)] if slots > 0 else []

    # MVO weights (relative split of deployable cash)
    names = [c["ticker"] for c in buy_sel]
    weights = []
    if names:
        rets = []
        for t in names:
            ser = dm[t]["Close"].loc[:signal_dt].pct_change().dropna().iloc[-cfg.volar_lookback:]
            rets.append(ser)
        R = pd.concat(rets, axis=1)
        R.columns = names
        R = R.dropna()
        if R.empty or R.shape[0] < max(20, int(0.4*cfg.volar_lookback)):
            weights = np.full(len(names), 1.0/len(names))
        else:
            mu, Sigma = R.mean().values, R.cov().values
            weights = markowitz_long_only(mu, Sigma)
    weights = np.array(weights) if len(names)>0 else np.array([])

    # Cash to deploy
    latest_prices = {t: float(dm[t].loc[signal_dt, "Close"]) for t in dm if signal_dt in dm[t].index}
    cash_avail = current_cash(cfg, pos, latest_prices)
    deploy_cash = cash_avail * cfg.deploy_cash_frac

    # Build BUY plan rows
    buy_rows = []
    for i, c in enumerate(buy_sel):
        w = float(weights[i]) if len(weights)>0 else (1.0/len(buy_sel) if buy_sel else 0.0)
        alloc = deploy_cash * (w / max(1e-9, weights.sum())) if len(weights)>0 else deploy_cash * w
        px = float(c["close_signal"])
        qty = int(math.floor(alloc / px))
        if qty <= 0:
            continue
        pct_52w = c["close_signal"] / c["high_52w"] if c["high_52w"]>0 else np.nan
        reason = (f'{c["signal_reason"]}; 52w%={pct_52w:.1%} (>= {CFG.within_pct_of_52w_high:.0%}); '
                  f'VOLAR rank {i+1}/{len(buy_sel)} (VOLAR={c["volar"]:.2f}); '
                  f'MVO weight={w:.1%} of capped cash ({CFG.deploy_cash_frac:.0%})')
        buy_rows.append({
            "action": "BUY",
            "ticker": c["ticker"],
            "signal_date": str(signal_dt.date()),
            "planned_exec_date": str(exec_dt.date()),
            "reason": reason,
            "suggest_qty": qty,
            "ref_price_signal": px,
            "alloc_inr": float(qty * px),
            "rsi": c["rsi"], "adx": c["adx"],
            "ema_fast": c["ema_fast"], "ema_slow": c["ema_slow"], "ema_htf": c["ema_htf"],
            "volar": c["volar"]
        })

    # Write outputs
    stamp = pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y%m%d_%H%M%S")
    plan_path = os.path.join(cfg.out_dir, f"plan_{stamp}.csv")
    pd.DataFrame(sell_rows + buy_rows).to_csv(plan_path, index=False)
    log.info("Plan written: %s", plan_path)

    cands_path = os.path.join(cfg.out_dir, f"candidates_{stamp}.csv")
    pd.DataFrame(cand_ranked).to_csv(cands_path, index=False)
    log.info("Candidates written: %s", cands_path)

    # Update positions ledger (virtual) → use planned_exec_date for buys
    exited = {r["ticker"] for r in sell_rows}
    pos = pos[~pos["ticker"].isin(exited)].copy()
    for r in buy_rows:
        pos = pd.concat([pos, pd.DataFrame([{
            "ticker": r["ticker"],
            "entry_date": pd.to_datetime(r["planned_exec_date"]),
            "entry_price": r["ref_price_signal"],
            "shares": int(r["suggest_qty"])
        }])], ignore_index=True)
    save_positions(pos, cfg)
    log.info("Positions updated: %s", cfg.positions_csv)

    # Telegram alert (optional) — includes top candidates
    if cfg.telegram_enabled and cfg.telegram_bot_token and cfg.telegram_chat_id:
        try:
            msg_lines = []
            msg_lines.append(f"📈 Daily Plan | signal={signal_dt.date()} → exec={exec_dt.date()}")
            if sell_rows:
                msg_lines.append("\nSELL:")
                for s in sell_rows:
                    msg_lines.append(
                        f"- {s['ticker']}: {s['reason']} qty={s['exit_qty']} "
                        f"(stop={s['ref_stop_level']:.2f}, tgt={s['ref_target_level']:.2f})"
                    )
            if buy_rows:
                msg_lines.append("\nBUY (for next open):")
                for b in buy_rows:
                    msg_lines.append(f"- {b['ticker']}: qty {b['suggest_qty']} @ {b['ref_price_signal']:.2f}")
            if cand_ranked:
                msg_lines.append("\nTOP CANDIDATES:")
                for row in cand_ranked[:5]:
                    msg_lines.append(f"- {row['rank']:>2}. {row['ticker']} | VOLAR={row['volar']:.2f} | 52w%={row['pct_of_52w_high']:.1%}")
            if not (sell_rows or buy_rows or cand_ranked):
                msg_lines.append("No actions.")
            text = "\n".join(msg_lines)
            url = f"https://api.telegram.org/bot{cfg.telegram_bot_token}/sendMessage"
            payload = {"chat_id": cfg.telegram_chat_id, "text": text}
            r = requests.post(url, json=payload, timeout=10)
            if r.status_code == 200:
                log.info("Telegram alert sent.")
            else:
                log.warning("Telegram send failed: %s %s", r.status_code, r.text)
        except Exception as e:
            log.warning("Telegram error: %s", e)

    return plan_path, cands_path

# --------------------------
# Main
# --------------------------
def main():
    global APPLY_FEES
    APPLY_FEES = bool(CFG.apply_fees)
    ensure_dirs(CFG.cache_dir, CFG.data_dir, CFG.out_dir)
    plan_path, cands_path = build_daily_plan(CFG)
    log.info("Done. Plan: %s | Candidates: %s", plan_path, cands_path)

if __name__ == "__main__":
    main()


2025-10-08 16:59:50 | ERROR | HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ^CNX500"}}}
2025-10-08 16:59:50 | ERROR | 
1 Failed download:
2025-10-08 16:59:50 | ERROR | ['^CNX500']: YFTzMissingError('possibly delisted; no timezone found')
2025-10-08 16:59:50 | INFO | Using benchmark: ^CRSLDX
2025-10-08 16:59:50 | INFO | Universe size with data: 500 | Benchmark: ^CRSLDX
2025-10-08 16:59:52 | INFO | Plan written: plans/plan_20251008_165952.csv
2025-10-08 16:59:52 | INFO | Candidates written: plans/candidates_20251008_165952.csv
2025-10-08 16:59:52 | INFO | Positions updated: data/positions.csv
2025-10-08 16:59:52 | INFO | Done. Plan: plans/plan_20251008_165952.csv | Candidates: plans/candidates_20251008_165952.csv
